In [ ]:
%pip install datasets transformers
%pip install evaluate
%pip install bert_score
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm


In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from datasets import load_dataset
from torchtext.datasets import multi30k, Multi30k
from torch import Tensor
import torch
import evaluate as E
import torch.nn as nn
from torch.nn import Transformer
import math
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import T5Tokenizer, T5Model
import numpy as np



In [3]:
val_data = load_dataset("wmt16", "de-en", split="validation")
test_data = load_dataset("wmt16", "de-en", split="test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/4548885 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2169 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2999 [00:00<?, ? examples/s]

In [16]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
model_name = "t5-small"
model_2b = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer_2b = T5Tokenizer.from_pretrained(model_name)

model_2b = model_2b.to(DEVICE)


cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
bleu = E.load("bleu")
meteor_metric = E.load("meteor")
bertscore_metric = E.load("bertscore")

def translate_2b(sentence):
    prefix = "translate English to German: "
    input_text = prefix + sentence
    input_ids = tokenizer_2b.encode(input_text, return_tensors="pt")
    input_ids = input_ids.to(DEVICE)
    output_ids = model_2b.generate(input_ids, max_length=100, num_beams=4, early_stopping=True)
    translated_sentence = tokenizer_2b.decode(output_ids[0], skip_special_tokens=True)
    return translated_sentence

def test_metrics_2b(data):
  references = []
  predictions = []
  for id, pair in enumerate(data):
      # print(id,"/", len(data))

      tgt_sample = pair['translation']['de']
      src_sample = pair['translation']['en']

      predictions.append(translate_2b(src_sample))
      references.append([tgt_sample])

  bleu_score = bleu.compute(predictions=predictions, references=references)
  meteor_score = meteor_metric.compute(predictions=predictions, references=references)
  bertscore_score = bertscore_metric.compute(predictions=predictions, references=references, lang="de")

  bs_pre = np.mean(np.array(bertscore_score['precision']))
  bs_recall = np.mean(np.array(bertscore_score['recall']))
  bs_f1 = np.mean(np.array(bertscore_score['f1']))


  print(f'BLEU Score: {bleu_score}')
  print(f'METEOR Score: {meteor_score}')
  print(f'BERTScore-precision: {bs_pre}')
  print(f'BERTScore-recall: {bs_recall}')
  print(f'BERTScore-f1: {bs_f1}')
  # return bleu_score, meteor_score, bertscore_score


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [30]:
model_2b.eval()
print("2B-Val Metrics")
test_metrics_2b(val_data)


2B-Val Metrics
BLEU Score: {'bleu': 0.2823125666465826, 'precisions': [0.5945855614973262, 0.3374072253049566, 0.21694413614502406, 0.14595002996118073], 'brevity_penalty': 1.0, 'length_ratio': 1.0140081337550837, 'translation_length': 44880, 'reference_length': 44260}
METEOR Score: {'meteor': 0.5515488896080301}
BERTScore-precision: 0.8656165647363817
BERTScore-recall: 0.8643085971412817
BERTScore-f1: 0.8647710810118746


In [31]:
print("2B-Test Metrics")
test_metrics_2b(test_data)

2B-Test Metrics
BLEU Score: {'bleu': 0.3268254810286919, 'precisions': [0.6340116232529098, 0.38745753590528403, 0.26043603775924057, 0.18072829969887969], 'brevity_penalty': 0.9966754605913349, 'length_ratio': 0.9966809746445611, 'translation_length': 62461, 'reference_length': 62669}
METEOR Score: {'meteor': 0.5902469429945494}
BERTScore-precision: 0.8778189009290888
BERTScore-recall: 0.8741395660502468
BERTScore-f1: 0.8757985283152665
